In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import os
import csv

chrome = webdriver.Chrome()

In [ ]:
def salvar_usuario(dic_usuario):
    df = pd.DataFrame([dic_usuario])
    
    # Caminho do arquivo
    caminho = './usuarios.csv'

    # Se o arquivo não existir, escrevemos com o cabeçalho
    # Se já existir, apenas adicionamos os dados sem repetir o cabeçalho (header=False)
    if not os.path.exists(caminho):
        df.to_csv(caminho, index=False, mode='w', header=True, encoding='utf-8', sep='|', quotechar='"', quoting=csv.QUOTE_ALL)
    else:
        df.to_csv(caminho, index=False, mode='a', header=False, encoding='utf-8', sep='|', quotechar='"', quoting=csv.QUOTE_ALL)

In [ ]:
# Desafio 02
# https://curso-web-scraping.pages.dev/#/desafio/2
# server local
pagina = 'http://localhost:8000/#/desafio/2'
chrome.get(pagina)

Roteiro  
1. Ler arquivo de nomes  
1. para cada nome  
1. limpar campo busca  
1. preenche campo busca = `<input type="text" required="" placeholder="Procurar usuários..." class="px-4 grow text-gray-700">`  
1. clicar e buscar = `<button class="text-white bg-cyan-500 px-4 rounded-md disabled:bg-slate-300 disabled:cursor-not-allowed" style="pointer-events: all;"><i class="fi fi-rr-search"></i></button>`
1. aguardar carregar lista  
1. na lista de resultados = `<div class="users-list flex flex-wrap gap-8 mb-6 lg:mb-16 md:gap-4 max-h-[64rem] overflow-y-auto px-4 justify-center">`  
1. para cada resultado = `<div class="items-center bg-gray-50 rounded-lg shadow dark:bg-gray-900 dark:border-gray-700 max-w-[350px] w-full">`  
1. coletar informações:  
* Imagem = `<img class="w-full rounded-t-lg" src="https://avatars.githubusercontent.com/u/74846533" alt="Dr. Dalton Barros">`  
* Nome  = `<h3 class="text-xl font-bold tracking-tight text-gray-900 dark:text-white">Dr. Dalton Barros</h3>`  
* Profissão = `<span class="text-gray-500 dark:text-gray-400">Produtor</span>`  
* E-mail = `<li class="truncate max-w-[250px] "><span>E-mail</span>: dalton.barros@example.org</li>`  
* Telefone = `<li><span>Telefone</span>: (87) 99940-1116</li>`  
* Usuário = `<li class="truncate max-w-[250px] "><span>Usuário</span>: dalton_barros</li>`  
* Estado = `<li><span>Estado</span>: Amazonas</li>`  

In [ ]:
# Abre o arquivo para leitura
with open('./desafio_2.json', 'r', encoding='utf-8') as arquivo:
    # Converte o conteúdo JSON em um objeto Python (dicionário/lista)
    lista_dados = json.load(arquivo)
print(lista_dados)

['Dalton', 'Ana', 'Maria', 'João', 'Leonardo', 'Talita', 'Rafael']


In [ ]:
for nome in lista_dados:
    #mapear elementos
    busca = chrome.find_element(By.CSS_SELECTOR, 'input[type="text"]')
    btn_busca = chrome.find_element(By.CSS_SELECTOR, 'button.text-white')
    # print(btn_busca.get_property('className'))
    #
    busca.clear()
    busca.send_keys(nome)
    btn_busca.click()
    # aguardar carregar
    # Configurar Espera de elementos da pagina
    tempo_max = 20  # tempo maximo de espera
    tempo_tentativa = 1  # a cada segundo
    wait = WebDriverWait(driver=chrome, timeout=tempo_max, poll_frequency=tempo_tentativa)
    #aguadar divs de usuario
    div = wait.until(
        EC.presence_of_element_located(locator=(By.CSS_SELECTOR, 'div.users-list')))
    # print(div.get_attribute('class'))

    divs_usuarios = chrome.find_elements(By.CSS_SELECTOR, 'div.users-list > div.items-center')
    for usuario in divs_usuarios:
        imagem = usuario.find_element(By.CSS_SELECTOR, 'img').get_attribute('src').strip()
        # print(imagem.get_attribute('alt'), imagem.get_attribute('src') )
        nome = usuario.find_element(By.CSS_SELECTOR, 'h3.text-xl').text.strip()
        # print(nome)
        profissao = usuario.find_element(By.CSS_SELECTOR, 'span.text-gray-500').text.strip()
        # print(profissao)
        email = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)').text.split(":")[1].strip()
        # print(email)
        telefone = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)').text.split(":")[1].strip()
        # print(telefone) li class="truncate
        username = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)').text.split(":")[1].strip()
        # print(username) ul > li:nth-child(4)
        uf = usuario.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)').text.split(":")[1].strip()
        #print(uf)

        dados_usuario = {
            "url": imagem,
            "nome": nome,
            "profissao":profissao,
            "email":email,
            "telefone":telefone,
            "username":username,
            "uf":uf
        }
        # Salvar em CSV
        salvar_usuario(dados_usuario)

print('Finalizou')
chrome.close()

Finalizou
